In [1]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
from ctrace.constraint import ProbMinExposedMIP, ProbMinExposed
from ctrace.utils import PQ_deterministic, find_contours, find_excluded_contours
from ctrace.dataset import load_graph, load_sir
from ctrace.solve import basic_non_integer_round
import numpy as np
import time

In [3]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver("GUROBI")
print(solver)

<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x7f1028d6a2a0> >


In [4]:
G = load_graph("montgomery")

In [5]:
# Load JSON t7.json
SIR = load_sir('t7.json')

I = set().union(*SIR["I_Queue"])
R = SIR["R"]

In [6]:
# Set K value
K = 300
contour1, contour2 = find_excluded_contours(G, I, R)
p1, q = PQ_deterministic(G, I, contour1, p=0.08)

In [7]:
sol_lp = ProbMinExposed(G, I, contour1, contour2, p1, q, K)
sol_lp.solve_lp()

In [65]:
print(sol_lp.objectiveVal)
print(sol_lp.isOptimal)

4044.133739955364
True


In [66]:
sol = ProbMinExposedMIP(G, I, contour1, contour2, p1, q, K, costs=None, solver='GUROBI')
sol.solve_lp()

In [67]:
import collections, numpy as np
print(sol.objectiveVal)
sol_vec = sol.getVariables()
collections.Counter(sol_vec)

4056.170718780131


Counter({0.0: 24357, 1.0: 300})

In [68]:
sol.isOptimal

True

In [72]:
sol_lp = ProbMinExposed(G, I, contour1, contour2, p1, q, K)
rounding_sol = basic_non_integer_round(sol_lp)


In [73]:
rounding_sol[0]

4067.0998905092974